# Import

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import random
from typing import List, Union
from joblib import Parallel, delayed

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.model_selection import KFold

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import datetime
import pytz
import matplotlib.pyplot as plt

# Config

In [ ]:
config = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'base_path': 'open (1)', # change relative path of data -> ex) kaggle: '/kaggle/input/swdacon', colab: '/content/drive/MyDrive/open'
    'save_path': 'result', # change relative path where result is stored -> ex) kaggle: '/kaggle/working/', colab: '/content/drive/MyDrive/open'
    'train_data': 'train.csv', # change train data csv name
    'test_data': 'test.csv', # change test data csv name
    'seed': 42,
    'valid_size': 0.1,
    'early_stopping': 30,
    'scheduler': True,
    'nums_pixel': False,
    'k-fold': 1, # 1이하의 정수이면 사용하지 않음
    'train' : {
       'batch_size' : 16,
       'num_workers': 1,
       'epochs': 200,
       'lr': 0.001,
    },
    'inference' : {
       'batch_size' : 8,
       'num_workers': 1,
       'threshold': 0.35,
    },
}

custom_transform = {
    'train':A.Compose([
        A.augmentations.crops.transforms.RandomCrop(224,224,p=1.0),
        A.RandomRotate90(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(),
        ToTensorV2()
    ]),
    'valid':A.Compose([
        A.augmentations.crops.transforms.CenterCrop(224,224,p=1.0),
        A.Normalize(),
        ToTensorV2()
    ]),
    'test': A.Compose([
        A.Normalize(),
        ToTensorV2()
    ]),
}

In [ ]:
# Colab 사용시 활성화
# 드라이브 마운트

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
if not os.path.exists(config['save_path']):
    os.makedirs(config['save_path'])
    print("save path가 생성되었습니다.")
print("save_path:", os.path.abspath(config['save_path']))
if not os.path.exists(config['base_path']):
    raise FileNotFoundError("base path가 존재하지 않습니다. 데이터가 있는 경로를 확인해주세요.")
else:
    print("base_path:", os.path.abspath(config['base_path']))

# Utils

In [ ]:
# 시드 고정 함수
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

# RLE 디코딩 함수
def rle_decode(mask_rle: Union[str, int], shape=(224, 224)) -> np.array:
    '''
    mask_rle: run-length as string formatted (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    if mask_rle == -1:
        return np.zeros(shape)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# dice score 계산 함수
def dice_score(prediction: np.array, ground_truth: np.array, smooth=1e-7) -> float:
    '''
    Calculate Dice Score between two binary masks.
    '''
    intersection = np.sum(prediction * ground_truth)
    return (2.0 * intersection + smooth) / (np.sum(prediction) + np.sum(ground_truth) + smooth)

def calculate_dice_scores(validation_df, img_shape=(224, 224)) -> List[float]:
    '''
    Calculate Dice scores for a dataset.
    '''
    # Extract the mask_rle columns
    pred_mask_rle = validation_df.iloc[:, 3]
    gt_mask_rle = validation_df.iloc[:, 4]

    def calculate_dice(pred_rle, gt_rle):
        pred_mask = rle_decode(pred_rle, img_shape)
        gt_mask = rle_decode(gt_rle, img_shape)
        if np.sum(gt_mask) > 0 or np.sum(pred_mask) > 0:
            return dice_score(pred_mask, gt_mask)
        else:
            return None  # No valid masks found, return None
    dice_scores = [calculate_dice(pred_rle, gt_rle) for pred_rle, gt_rle in zip(pred_mask_rle, gt_mask_rle)]
    dice_scores = [score for score in dice_scores if score is not None]  # Exclude None values
    return np.mean(dice_scores)

def calculate_nums_pixel(validation_df, img_shape=(224, 224)):
    '''
    Validation의 건물 pixel 수와 Prediction의 건물 pixel 수를 계산합니다.
    더 많이 예측하는지, 덜 예측하는지 기준을 잡고 threshold를 조정에 도움이 될 수 있습니다.
    '''
    eps = 1e-6
    batch_temp, count = 0, 0
    more_pred,less_pred = 0, 0
    pred_mask = validation_df.iloc[:, 2]
    gt_mask = validation_df.iloc[:, 3]
    for p_mask, t_mask in zip(pred_mask, gt_mask):
        if np.sum(rle_decode(t_mask, img_shape)):
            count += 1
            temp = float(int(np.sum(rle_decode(t_mask, img_shape)) - int(np.sum(rle_decode(p_mask, img_shape)))) / (int(np.sum(rle_decode(t_mask, img_shape))) + eps))
            if temp > 0: more_pred+= 1
            elif temp < 0: less_pred+= 1
            batch_temp += temp
    return batch_temp/count, more_pred, less_pred

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

# Train
- 1024 * 1024 * 3 이미지 학습

### load train data

In [ ]:
train_df = pd.read_csv(f"{config['base_path']}/{config['train_data']}")
train_df['img_path'] = config['base_path'] + train_df['img_path'].str[1:]
if config['k-fold'] > 1:
    kfold = KFold(n_splits=config['k-fold'], random_state=config['seed'], shuffle=True)
    print(kfold)
else:
    train, val = train_test_split(train_df, test_size=config['valid_size'], random_state=config['seed'])
    print("train: ", len(train), "   valid: ", len(val))

### Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, mask_rles = None, transform=None, infer=False):
        self.img_paths = img_paths
        self.mask_rles = mask_rles
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths.iloc[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.mask_rles.iloc[idx]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

### Data Loader

In [ ]:
fix_seed(config['seed'])

if config['k-fold'] > 1:
    print(f"k-fold: {config['k-fold']} 입력되어, training할 때 fold별 dataloader가 생성됩니다.")
else:
    train_dataset = CustomDataset(img_paths=train['img_path'], mask_rles=train['mask_rle'], transform=custom_transform['train'])
    train_dataloader = DataLoader(train_dataset, batch_size=config['train']['batch_size'], shuffle=True)

    valid_dataset = CustomDataset(img_paths=val['img_path'], mask_rles=val['mask_rle'], transform=custom_transform['valid'])
    valid_dataloader = DataLoader(valid_dataset , batch_size=config['train']['batch_size'], shuffle=False)

### Model
- FCN 8s

In [ ]:
# !pip install torchfcn

### Load model

In [ ]:
if config['k-fold'] > 1:
    print(f"k-fold: {config['k-fold']} 입력되어, training할 때 fold별 model이 생성됩니다.")
else:
    model = torchfcn.models.FCN8s(n_class=21) # 21 classese로 학습된 pretrained weight
    model.load_state_dict(torch.load(torchfcn.models.FCN8s.download()))

In [ ]:
def our_task(n_class, fcn8s):
    fcn8s.score_fr = nn.Conv2d(4096, n_class, 1)
    fcn8s.score_pool3 = nn.Conv2d(256, n_class, 1)
    fcn8s.score_pool4 = nn.Conv2d(512, n_class, 1)
    fcn8s.upscore2 = nn.ConvTranspose2d(
            n_class, n_class, 4, stride=2, bias=False)
    fcn8s.upscore8 = nn.ConvTranspose2d(
            n_class, n_class, 16, stride=8, bias=False)
    fcn8s.upscore_pool4 = nn.ConvTranspose2d(
            n_class, n_class, 4, stride=2, bias=False)
    return fcn8s

model = our_task(1, model) # 우리 task는 class가 1

### Validation

In [ ]:
def validation(config, model, criterion, valid_loader, val):
    model.eval()
    valid_loss = 0
    result = []
    transformed_mask = []
    val_df = val.copy()

    with torch.no_grad():
        for images, masks in tqdm(valid_loader):
            if type(transformed_mask) == torch.Tensor:
                transformed_mask = torch.cat([transformed_mask, masks])
            else:
                transformed_mask = masks.clone().detach()
            images = images.float().to(config['device'])
            masks = masks.float().to(config['device'])

            outputs = model(images)
            loss = criterion(outputs, masks.unsqueeze(1))
            valid_loss += loss.item()

            output_masks = torch.sigmoid(outputs).cpu().numpy()
            output_masks = np.squeeze(output_masks, axis=1)
            output_masks = (output_masks > config['inference']['threshold']).astype(np.uint8)

            for i in range(len(images)):
                mask_rle = rle_encode(output_masks[i])
                if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                    result.append(-1)
                else:
                    result.append(mask_rle)
        val_df['valid_mask_rle'] = result
        val_df['transformed_mask_rle'] = list(map(rle_encode, transformed_mask.squeeze().numpy()))
        dice_score = calculate_dice_scores(val_df)
        if config['nums_pixel']:
            mean_error_ratio, more_pred, less_pred = calculate_nums_pixel(val_df)
    if config['nums_pixel']:
        return valid_loss/len(valid_loader), dice_score, mean_error_ratio, more_pred, less_pred
    else:
        return valid_loss/len(valid_loader), dice_score

### Train

In [ ]:
def training(config, model, train_loader, valid_loader, val, fold=0):
    model = model.to(config['device'])
    es_count = 0
    min_val_loss = float('inf')
    best_model = None

    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['train']['lr'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-8, verbose=True)
    print("***TRAINING START***")
    # training loop
    for epoch in range(config['train']['epochs']):
        model.train()
        epoch_loss = 0
        for images, masks in tqdm(train_loader):
            images = images.float().to(config['device'])
            masks = masks.float().to(config['device'])

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks.unsqueeze(1))
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if config['nums_pixel']:
            val_loss, dice_score, mean_error_ratio, more_pred, less_pred  = validation(config, model, criterion, valid_loader, val)
        else:
            val_loss, dice_score = validation(config, model, criterion, valid_loader, val)

        es_count += 1
        if min_val_loss > val_loss:
            es_count = 0
            min_val_loss = val_loss
            best_model = model
            state_dict = model.state_dict()
            best_epoch = epoch + 1
            print(f"Epoch [{epoch + 1}] New Minimum Valid Loss!")
            if epoch+1 > 50: # 비정상적인 종료에 대비해 몇 epoch 이상부터 저장할지 결정합니다.
                if fold:
                    current_model = f"fold{fold}_current_best_model.pt"
                else:
                    current_model = "current_best_model.pt"
                print("..save current best model..")
                torch.save(state_dict, f'{config["save_path"]}/{current_model}')

        if config['scheduler']:
            scheduler.step(val_loss)

        if es_count == config['early_stopping']:
            if config['nums_pixel']:
                print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, ES Count: {es_count}')
                print(f'Dice Coefficient: {dice_score:6f}, (GT - Pred)/GT: {mean_error_ratio:2f}, More Pred : Less Pred = {more_pred} : {less_pred}')
            else:
                print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, Dice Coefficient: {dice_score:6f}, ES Count: {es_count}')
            print(f"Early Stopping Count에 도달하지 않았습니다! \nEarly Stopping Count: {config['early_stopping']} Best Epoch: {best_epoch}")
            print("***TRAINING DONE***")
            return best_model, state_dict

        if config['nums_pixel']:
            print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, ES Count: {es_count}')
            print(f'Dice Coefficient: {dice_score:6f}, (GT - Pred)/GT: {mean_error_ratio:2f}, More Pred : Less Pred = {more_pred} : {less_pred}')
        else:
            print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, Dice Coefficient: {dice_score:6f}, ES Count: {es_count}')
        print("------------------------------------------------------------------------------------")

    print(f"Early Stopping Count에 도달하지 않았습니다! \nEarly Stopping Count: {config['early_stopping']} Best Epoch: {best_epoch}")
    print("***TRAINING DONE***")
    return best_model, state_dict

In [ ]:
torch.cuda.is_available() # 학습 전에 GPU 쓰고 있나 확인

In [ ]:
fix_seed(config['seed'])

if config['k-fold'] > 1:
    best_models, best_model_state_dicts = [], []
    for i, (train_idx, val_idx) in enumerate(kfold.split(train_df)):
        train = train_df.iloc[train_idx,:]
        val = train_df.iloc[val_idx,:]
        train_dataset = CustomDataset(img_paths=train['img_path'], mask_rles=train['mask_rle'], transform=custom_transform['train'])
        train_dataloader = DataLoader(train_dataset, batch_size=config['train']['batch_size'], shuffle=True)

        valid_dataset = CustomDataset(img_paths=val['img_path'], mask_rles=val['mask_rle'], transform=custom_transform['valid'])
        valid_dataloader = DataLoader(valid_dataset , batch_size=config['train']['batch_size'], shuffle=False)
        print(f"--- Start Fold {i + 1} ---")
        model = torchfcn.models.FCN8s(n_class=21) # 21 classese로 학습된 pretrained weight
        model.load_state_dict(torch.load(torchfcn.models.FCN8s.download()))
        model = our_task(1, model).to(config['device'])
        best_model, best_model_state_dict = training(config, model, train_dataloader, valid_dataloader, val, i+1)
        best_models.append(best_model)
        best_model_state_dicts.append(best_model)
        print(f"---- End Fold {i + 1} ----")
else:
    best_model, best_model_state_dict = training(config, model, train_dataloader, valid_dataloader, val)

## Inference
- 224 * 224 * 3 이미지 추론

### load test data

In [ ]:
test_df = pd.read_csv(f"{config['base_path']}/{config['test_data']}")
test_df['img_path'] = config['base_path'] + test_df['img_path'].str[1:]

### Data Loader

In [ ]:
fix_seed(config['seed'])

if config['k-fold'] > 1:
    print(f"k-fold: {config['k-fold']} 입력되어, inference할 때 fold별 dataloader가 생성됩니다.")
else:
    test_dataset = CustomDataset(img_paths=test_df['img_path'], transform=custom_transform['test'], infer=True)
    test_dataloader = DataLoader(test_dataset, batch_size=config['inference']['batch_size'], shuffle=False)

In [ ]:
def inference(config, model, test_loader):
    with torch.no_grad():
        model.eval()
        result = []
        for images in tqdm(test_loader):
            images = images.float().to(config['device'])

            outputs = model(images)
            masks = torch.sigmoid(outputs).cpu().numpy()
            masks = np.squeeze(masks, axis=1)
            masks = (masks > config['inference']['threshold']).astype(np.uint8)

            for i in range(len(images)):
                mask_rle = rle_encode(masks[i])
                if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                    result.append(-1)
                else:
                    result.append(mask_rle)
    return result

In [ ]:
if config['k-fold'] > 1:
    inference_results = []
    for i, fold_model in enumerate(best_models):
        test_dataset = CustomDataset(img_paths=test_df['img_path'], transform=custom_transform['test'], infer=True)
        test_dataloader = DataLoader(test_dataset, batch_size=config['inference']['batch_size'], shuffle=False)
        print(f"--- Start Fold {i + 1} ---")
        inference_result = inference(config, fold_model, test_dataloader)
        inference_results.append(inference_result)
else:
    inference_result = inference(config, best_model, test_dataloader)

## Submission

In [ ]:
kst = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(tz=kst)
current_time = now.strftime("%y%m%d-%H_%M_%S")

if config['k-fold'] > 1:
    for i,inference_result in enumerate(inference_results):
        submit = pd.read_csv(f"{config['base_path']}/sample_submission.csv")
        submit['mask_rle'] = inference_result
        file_name = f"fold_{i+1}_{current_time}.csv"
        submit.to_csv(f"{config['save_path']}/{file_name}", index=False)

    # 모델 저장
    for i,best_model_state_dict in enumerate(best_model_state_dicts):
        model_name = f"fold_{i+1}_{current_time}.pt"
        torch.save(best_model, f"{config['base_path']}/{model_name}")
else:
    submit = pd.read_csv(f"{config['base_path']}/sample_submission.csv")
    submit['mask_rle'] = inference_result
    file_name = f"{current_time}.csv"
    submit.to_csv(f"{config['save_path']}/{file_name}", index=False)

    # 모델 저장
    model_name = f"{current_time}.pt"
    torch.save(best_model, f"{config['save_path']}/{model_name}")
print(file_name)
print(model_name)

# Submission Viewer
- fold 사용시엔 사용할 필요 x

In [ ]:
def submission_viewer(test_csv, submit_csv, img_num):
    """
    white -> 건물 black -> 배경
    1. Local에서 사용 시 test_csv, submit_csv, img_num만 입력
    2. colab에서 사용 시 아래 사항을 입력
    base_path = colab_base
    is_colab = True
    """
    mask_rle = submit_csv.iloc[img_num, 1]
    image_path = test_csv.iloc[img_num, 1]
    image = cv2.imread(image_path)
    mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(image)
    ax1.set_title('image')
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(mask,cmap='gray')
    ax2.set_title('mask')
    plt.show()

In [ ]:
if config['k-fold'] > 1:
    print("Ensemble을 진행하십시오.")
else:
    last_submit = pd.read_csv(f"{config['save_path']}/{file_name}")
    submission_viewer(test_df, last_submit, 0)
    submission_viewer(test_df, last_submit, 1)
    submission_viewer(test_df, last_submit, 2)